## API and SQL Exercises

The geoplace_key for Mexico is 3996063 (you can look this up using the /lookup/location end point).  Using the /list/grants apibeta endpoint, add up the total amount of the top 10 grants to recipients in Mexico. 

In [54]:
import requests
import json as js

s = requests.Session()
s.auth = ([provided in class])

r = s.get('https://apibeta.foundationcenter.org/v2.0/list/grants?location=3996063&year=&subject=&population=&support=&transaction=&recip_id=&funder_id=&sort_by=amount&sort_order=desc&format=json')

obj = js.loads(r.text)

total = 0
for row in obj['data']['results']:
    total += row['amount_usd']
print(total)
    
my_lst = [row['amount_usd'] for row in obj['data']['results']]  
print(sum(my_lst))

705423747
705423747


Supposed we wanted to get place identifiers mentioned in the grantmaker limitations text, for grantmakes located in Brooklyn.  You can get this text from the fc_sample database with a query like the following:

SELECT a.fcdbwin_gm_key, 
       a.NAME, 
       b.description 
FROM   grantmaker a 
       INNER JOIN long_text b 
               ON a.gm_key = b.gm_key 
WHERE  b.text_type = 'LM' 
       AND a.state = 'NY' 
       AND a.county = 'Kings' 
       
The api call has this format:

https://apibeta.foundationcenter.org/v2.0/text/locations?text=[text]

Write a program that gets the unique geoplace_keys referenced in this text for those grantmakers.

Supposed we wanted to get a sense of how grant funding is distributed among SDGs (sustainable development goals) for smal Ford Foundation grants in 2011.  ApiBeta has an endpoint that does this, text/taxonomies.  Here is a sample API call:

https://apibeta.foundationcenter.org/v2.0/text/taxonomies?text=[text]&taxonomy=sdg&threshold=60&chunk_text=false
 
We can get the grant description texts using a query something like this:

SELECT grant_key, 
       description, 
       amount 
FROM   grants a 
       INNER JOIN grantmaker b 
               ON a.gm_link = b.gm_key 
WHERE  b.fcdbwin_gm_key = 'FORD010' 
       AND Year(a.date_issued) = 2011 
       AND amount < 50000 
       AND Len(Isnull(description, '')) > 100 

Using the python features we've covered in class about dictionaries, sql, and web services, write a program that gets the grant descriptions and amounts from the database, gets the SDG goal codes based on the grant text, and sums up the grant amounts per SDG goal.

## Answers

From our class session on these problems - note that this code won't run within a notebook - you will need to create the geo file and call it from your main python code outside of the notebook, e.g., within pycharm.

In [ ]:
# Contents of geo.py:

import requests
import json

s = requests.Session()
s.auth = ('[provided in class]', '[provided in class]')

def anything(item):

    r = s.get('https://apibeta.foundationcenter.org/v2.0/text/locations?text=' + item)

    obj = json.loads(r.text)
    lst = obj["data"]["results"]
    x = []

    for i in range(len(lst)):
        y = lst[i]["geoplace_key"]
        z = lst[i]["name"]
        r = (y, z)
        x.append(r)

    return x

In [ ]:
# Content of main script file:

import pyodbc
import geo

cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=172.16.7.121;DATABASE=fc_sample;uid=python_class_user;pwd=[provided in class]")
sql = "SELECT top 10 a.fcdbwin_gm_key, a.NAME, b.description FROM grantmaker a INNER JOIN long_text b ON a.gm_key = b.gm_key WHERE b.text_type = 'LM' AND a.state = 'NY' AND a.county = 'Kings'"
cursor = cnxn.cursor()
limitation = []

for row in cursor.execute(sql):
    limitation.append(row[2])

cursor.close()
q = []

for item in limitation:
    x = geo.anything(item)
    q.append(x)
pair = zip(q, limitation)

print(list(pair))


## Word Count and Grantmaker Similarity

The challenge is to write a program that reads the grantmaker mission statements, and finds the most similar grantmaker based on their mission statements.  We can judge similarity by looking at the words that the mission statements have in common. 

To keep this manageable, let's just look at the grantmakers in California, and see which one is the most similar to PETCO Foundation (PETC003) based in their mission statements.

    SELECT a.gm_key
         ,NAME
         ,description purpose
    FROM grantmaker a
    INNER JOIN long_text b ON a.gm_key = b.gm_key
    WHERE IsNull(b.text_type, '') = 'PU'
    amd state = 'CA'

We can do this by creating a dictionary where each grantmaker has a key, and each dictionary is a dictionary of the words in the mission statement along with its count.

In the simplest case, we can try to which is the most similar to PETCO Foundation by looking at each of the words in our vocabulary (stored in a separate set), and incremening a per-grantmaker score by each word found in common.  

Then we can sort the results by score descending and seeing which grantmaker is the most similar.

What is the most similar grantmaker according to this method?

One possible improvement is to weight their score by penalizing words that are common, and weighting higher those words that are rare.  One easy was to do this is by multiplying the score by a weight calculated by multiplying these two terms together: 

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

IDF(t) = log_e(Total number of documents / Number of documents with term t in it).

tf-idf weight = TF * IDF, for each term

### Here is some starting code:

In [1]:
import pyodbc
import math

cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=172.16.7.121;DATABASE=fc_sample;uid=[provided in class];pwd=[provided in class]")
sql = """
    SELECT a.fcdbwin_gm_key
         ,description purpose
    FROM grantmaker a
    INNER JOIN long_text b ON a.gm_key = b.gm_key
    WHERE IsNull(b.text_type, '') = 'PU'
    and state = 'CA'
    """

cursor = cnxn.cursor()
gms = dict()
vocabulary = set()

for row in cursor.execute(sql):
    # Create a word count dictionary for each grantmakers
    # Add each word to the vocabulary

cursor.close()

# Get the counts for Petco
source_counts = gms['PETC003']
scores = dict()
for k in gms.keys():
    if k == 'PETC003':
        continue
    # For each vocabulary term, increase the score for those that appear in both
    word_counts = gms[k]

# Sort the scores descending and see which is closest to Petco.
source_counts = gms['PETC003']
scores = dict()
counter = 0
for k in gms.keys():
    if k == 'PETC003':
        continue
    word_counts = gms[k]
    score = 0.0
    for v in vocabulary:
        s1 = source_counts.get(v, 0)
        s2 = word_counts.get(v, 0)
        score += (s1 * s2) * math.log(len(gms) / doc_counts[v])
    counter += 1
    if counter % 100 == 0:
        print (counter, "Processed")
    scores[k] = score

num_docs = len(gms.keys())
d = sorted(scores.items(), key=lambda x: x[1], reverse=True)
print (d[0])



IndentationError: expected an indented block (<ipython-input-1-16202c70dedf>, line 22)